In [1]:
import pandas as pd
import random
from itertools import chain
import pickle
import nltk

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

def clean_word_f(x):
    out = []
    x = word_tokenize(x.lower())
    for i in x:
        i = i.strip("'")
        if i not in stop_words and i not in subs_list and re.match(r'[a-z]', i) and len(i) >2:
            out.append(wnl.lemmatize(i, get_pos(i)))
    return out

In [2]:
df = pickle.load(open("dataframe.pickle", "rb" ) )

## Training a classifyer
Big thanks to this tutorial, which I've shamelessly robbed from:

https://pythonprogramming.net/words-as-features-nltk-tutorial/

In [40]:
df.token_sent
# df.substance
# Keeping only the experiences clearly listed as positive or negative. Might bring the other ones in again later, but for now
# this is all I need.
# df = df[df['tone'].isin(["Positive", "Negative"])]

# ## I want at least 100 of each substance and category
# for i in ['substance', 'sub_cats']:
#     top_list = df[i].value_counts().reset_index()
#     df = df[df[i].isin(top_list[top_list[i] > 100]['index'])]

id
1         ['\r\nfinally friday arrives, i bought my 30$ ...
2         ['\r\nat first it was just another rave with m...
3         ["\r\ni've rolled about 10 times before this e...
4         ['\r\n\r\ni have had the opportunity to try 2c...
5         ['\r\n\r\nhaving read some accounts of nausea ...
6         ['\r\n\r\nthis was not my first experiment wit...
7         ['\r\n\r\nmy first experience with the mushroo...
8         ['\r\n\r\ntripping outdoors is always the best...
9         ['\r\n\r\nthis was going to be my second time ...
10        ['\r\n\r\nmy friend had some experience with x...
11        ["\r\n\r\nthe only drug other than pot that i'...
12        ['\r\n\r\necstacy was first brought to my atte...
13        ["\r\n\r\ni had done salvia a few times before...
14        ["\r\n\r\n\r\nmy friend 'bill' and i were goin...
15        ['\r\n\r\ni unfortunately happen to be one tha...
16        ['\r\nmy family owns a cottage in the muskoka ...
17        ['\r\n\r\nwhat i had really

In [6]:
# Create a list of tuples from the df, thanks to https://stackoverflow.com/questions/9758450/pandas-convert-dataframe-to-array-of-tuples
documents = list(df[['word_clean','tone']][df['tone'].isin(['Positive', 'Negative'])].itertuples(index=False, name=None))

## Shuffle for good measure
random.shuffle(documents)

## Create a big ole list of all words in the corpus for a freq count
all_words = list(chain.from_iterable(df['word_clean'][df['tone'].isin(['Positive', 'Negative'])].tolist()))

## Word counts, should do something fun with this if I've got time
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:2000]

## Returns a featureset as a list of tuples of dictionaries
def find_features(documents):
    words = set(documents)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

## This does the same thing, but word_tokenizes its input. This is used for mucking around with new data.
def find_feats(document):
    words = clean_word_f(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

## And finally preparing our training and testing sets
split_len = int(round(len(featuresets) * .90, 0))
split_len

## Training and testing sets
training_set = featuresets[:split_len]
testing_set = featuresets[split_len:]


'['

In [7]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(10)

save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

Classifier accuracy percent: 71.95402298850576
Most Informative Features
                softness = True           Positi : Negati =     13.6 : 1.0
                  enigma = True           Positi : Negati =     11.3 : 1.0
               enrapture = True           Positi : Negati =     11.3 : 1.0
                hospital = True           Negati : Positi =     10.0 : 1.0
                 backrub = True           Positi : Negati =      9.7 : 1.0
                   brook = True           Positi : Negati =      9.0 : 1.0
                scariest = True           Negati : Positi =      8.9 : 1.0
      interconnectedness = True           Positi : Negati =      8.2 : 1.0
                 oneness = True           Positi : Negati =      7.7 : 1.0
                    rosy = True           Positi : Negati =      7.4 : 1.0


In [ ]:

print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

for i in [MultinomialNB, BernoulliNB, LogisticRegression, SGDClassifier, SVC, LinearSVC, NuSVC]:
    i = SklearnClassifier(i())
    i.train(training_set)
#     i.show_most_informative_features(15)
    print("Accuracy percent:", (nltk.classify.accuracy(i, testing_set))*100)


# MNB_classifier = SklearnClassifier(MultinomialNB())
# MNB_classifier.train(training_set)
# print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

# BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
# BernoulliNB_classifier.train(training_set)
# print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

# LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
# LogisticRegression_classifier.train(training_set)
# print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

# SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
# SGDClassifier_classifier.train(training_set)
# print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

# SVC_classifier = SklearnClassifier(SVC())
# SVC_classifier.train(training_set)
# print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

# LinearSVC_classifier = SklearnClassifier(LinearSVC())
# LinearSVC_classifier.train(training_set)
# print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

# NuSVC_classifier = SklearnClassifier(NuSVC())
# NuSVC_classifier.train(training_set)
# print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

In [ ]:
with open("input.txt") as f:
    test = f.read()

# for i in [MultinomialNB, BernoulliNB, LogisticRegression, SGDClassifier, SVC, LinearSVC, NuSVC]:
classified = classifier.prob_classify(find_feats(test))
print(classified.prob('Negative'))

# sia.polarity_scores(test)

In [ ]:
accuracy = {"Whole Report" : df.vw_acc.mean(), "Cleaned and Tokenised" : df.vc_acc.mean(), "Sentance Average" : df.vs_acc.mean()}
accuracy_scores = pd.DataFrame(list(accuracy.items()), columns = ["Method", "Accuracy"]).set_index("Method")
accuracy_scores